# Нормализация текста перед токенизацией: тест с моделью sbert
В этом ноутбуке проверяем, влияет ли предварительная обработка текста на результат токенизации.
Тестируем наличие в "сыром" тексте:
* заглавных букв, 
* знаков препинания, 
* разного формата записи чисел (цифрами или буквами), 
* взаимозаменяемых букв "ё" и "е".

In [1]:
from transformers import AutoTokenizer, AutoModel
import torch

In [2]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [3]:
#Sentences we want sentence embeddings for
sentences = ['съешь еще этих мягких французских булок да выпей чаю', # "нормализованный" текст
             'Съешь еще этих Мягких Французских Булок да выпей Чаю', # добавлены только заглавные буквы
             'съешь еще этих мягких французских булок, да выпей чаю!', # только знаки препинания
             'съешь ещё этих мягких французских булок да выпей чаю', # только буква 'ё'
             'Съешь ещё этих мягких французских булок, да выпей чаю!', # все вместе
             'Тридцать восемь попугаев', # числа записаны текстом
             '38 попугаев'] # числа записаны цифрами


In [4]:
#Load AutoModel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_mt_nlu_ru")
model = AutoModel.from_pretrained("ai-forever/sbert_large_mt_nlu_ru")

tokenizer_config.json:   0%|          | 0.00/331 [00:00<?, ?B/s]

c:\Users\-\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\-\.cache\huggingface\hub\models--ai-forever--sbert_large_mt_nlu_ru. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

c:\Users\-\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [5]:
#Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=24, return_tensors='pt')

In [6]:
#Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

In [7]:
#Perform pooling. In this case, mean pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

In [9]:
emb_shape = sentence_embeddings.shape
emb_shape

torch.Size([7, 1024])

## Примеры 0-4: заглавные буквы, знаки препинания, 'ё'

In [13]:
sentence_embeddings_np = sentence_embeddings.numpy()

In [28]:
assert (sentence_embeddings_np[0] == sentence_embeddings_np[1]).all(), "Заглавные буквы влияют на токенизацию"
print('Заглавные буквы не влияют на токенизацию')

Заглавные буквы не влияют на токенизацию


In [29]:
assert (sentence_embeddings_np[0] == sentence_embeddings_np[2]).all(), "Знаки препинания влияют на токенизацию"
print('Знаки препинания не влияют на токенизацию')

AssertionError: Знаки препинания влияют на токенизацию

In [30]:
assert (sentence_embeddings_np[0] == sentence_embeddings_np[3]).all(), "Ё влияют на токенизацию"
print('Ё не влияют на токенизацию')

Ё не влияют на токенизацию


In [31]:
assert (sentence_embeddings_np[0] == sentence_embeddings_np[4]).all(), "Внесенные изменения влияют на токенизацию"
print('Внесенные изменения не влияют на токенизацию')

AssertionError: Внесенные изменения влияют на токенизацию

## Примеры 5-6: формат записи чисел

In [32]:
assert (sentence_embeddings_np[5] == sentence_embeddings_np[6]).all(), "Формат записи чисел влияет на токенизацию"
print('Формат записи чисел не влияет на токенизацию')

AssertionError: Формат записи чисел влияет на токенизацию

## Выводы

Модель sbert "из коробки" обрабатывает **заглавные буквы** и взаимозаменяемость **"ё"** и "е". 

А вот **знаки препинания** и **формат записи чисел** на результат токенизации влияют.

Датасет, с которым мы работаем в рамках хакатона, уже предобработан: только маленькие буквы, нет знаков препинания, нет букв "ё", все числа записаны текстом.

Однако это касается только train столбца (результата распознавания моделью ASR). В ручной разметке встречаются буквы "ё" и запятые. Также в ручной разметке встречается дублирование гласных для передачи эмоциональной окраски (например, "эээ" вместо "э"). Учитывая, что ручная разметка участвует в расчете WER для определения метки класса, часть уже проставленных классов может быть некорректной.

Если нужно расширить датасет или провести тестирование на данных не из тестовой части данного датасета, то стоит привести все данные к единому формату. Это также касается и ручной разметки, но в этом случае, вероятно, придется заново оценить метки классов для всего датасета.

#### Пример блока нормализации:

In [51]:
import string

def normalize_text(text: list) -> list:
    '''
    Функция принимает список преложений и возвращает список, во всех предложениях которого:
    1. Буквы приведены к нижнему регистру
    2. Буква "ё" заменена на "е"
    3. Знаки препинания удалены
    '''
    result = []
    for sentence in text:
        sentence = sentence.lower()
        sentence = sentence.replace('ё', 'е')
        sentence = sentence.translate(str.maketrans('', '', string.punctuation))
        result.append(sentence)
                                
    return result

In [ ]:
# ! pip install words2numsrus
'''
Для перевода текстовой записи чисел в цифры удобно использовать готовую библиотечку words2numsrus.
https://pypi.org/project/words2numsrus/ 
Мы переводим текстовую запись чисел в цифры, потому что обратное преобразование (цифр в текст) для русского языка - сложная задача из-за обилия словоформ и падежей.
'''

In [52]:
from words2numsrus import NumberExtractor

def replace_numbers(text: list) -> list:
    '''
    Функция принимает список предложений и возвращает список, во всех предложениях которого все числа записаны цифрами.
    '''
    result = []
    for sentence in text:
        extractor = NumberExtractor()
        result.append(extractor.replace_groups(sentence))
                                
    return result

In [53]:
test = normalize_text(sentences)
print(test)

['съешь еще этих мягких французских булок да выпей чаю', 'съешь еще этих мягких французских булок да выпей чаю', 'съешь еще этих мягких французских булок да выпей чаю', 'съешь еще этих мягких французских булок да выпей чаю', 'съешь еще этих мягких французских булок да выпей чаю', 'тридцать восемь попугаев', '38 попугаев']


In [54]:
replace_numbers(test)

['съешь еще этих мягких французских булок да выпей чаю',
 'съешь еще этих мягких французских булок да выпей чаю',
 'съешь еще этих мягких французских булок да выпей чаю',
 'съешь еще этих мягких французских булок да выпей чаю',
 'съешь еще этих мягких французских булок да выпей чаю',
 '38 попугаев',
 '38 попугаев']